## Correcting some columns without removing data

In [1]:
# Packages
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Defining the path for the dataframe
dataframe = 'C:/Users/Peter/py_projects/solar_stats/cdaw_cme_flare_ar_smart_database.p'

# Importing the dataframe 
df = pd.read_pickle(dataframe)

### GOES flux

In [3]:
# Splitting goes_class column into 2 so as to separate the class form the flux value

""" The pattern ([a-zA-Z]+)([^a-zA-Z]+) means match a group of letters: ([a-zA-Z]+) followed by a group 
    of non letters: ([^a-zA-Z]+) """

glux = df.flare_goes_class.str.extract('([a-zA-Z]+)([^a-zA-Z]+)', expand=True)
glux.columns = ['goes_class', 'goes_flux']

# Adding new class column into df 
df['goes_class'] = glux['goes_class']

In [4]:
# Want to see which catalogues the errors are coming from
glux['flare_type'] = df['flare_type'].values

# Setting the NaNs to zeros for the time being
glux = glux.fillna(0)

In [5]:
# Checking which values need correcting
print(glux.goes_flux[pd.to_numeric(glux.goes_flux, errors='coerce').isnull()])

5433      7,4
5434      7,4
5585     6.1*
5640     2.6*
5641     2.6*
         ... 
28147    2.3*
28330    2.8*
28568    1.4*
28675    1.4*
28723    1.1*
Name: goes_flux, Length: 331, dtype: object


In [6]:
check = glux[pd.to_numeric(glux.goes_flux, errors='coerce').isnull()]
check

goes_class goes_flux flare_type
5433           C       7,4       swpc
5434           C       7,4       swpc
5585           C      6.1*     rhessi
5640           M      2.6*     rhessi
5641           M      2.6*     rhessi
...          ...       ...        ...
28147          B      2.3*     rhessi
28330          B      2.8*     rhessi
28568          C      1.4*     rhessi
28675          C      1.4*     rhessi
28723          C      1.1*     rhessi

[331 rows x 3 columns]

In [7]:
check['flare_type'].value_counts().to_dict()

{'rhessi': 329, 'swpc': 2}

In [8]:
# Getting rid of the asterisks
glux.goes_flux = glux.goes_flux.replace('\*','',regex=True)

In [9]:
# Checking what else needs to be corrected
print(glux.goes_flux[pd.to_numeric(glux.goes_flux, errors='coerce').isnull()])

5433    7,4
5434    7,4
Name: goes_flux, dtype: object


In [10]:
# Okay, replacing the commas with periods
glux.goes_flux = glux.goes_flux.replace('\,','.',regex=True)

In [11]:
# Should be all?
print(glux.goes_flux[pd.to_numeric(glux.goes_flux, errors='coerce').isnull()])

Series([], Name: goes_flux, dtype: object)


In [12]:
# Converting to a float for future use
glux.goes_flux = glux.goes_flux.astype(float)

In [13]:
glux

goes_class  goes_flux flare_type
0              0        0.0          0
1              0        0.0          0
2              0        0.0          0
3              0        0.0          0
4              0        0.0          0
...          ...        ...        ...
30223          0        0.0          0
30224          0        0.0          0
30225          0        0.0          0
30226          0        0.0          0
30227          0        0.0          0

[30228 rows x 3 columns]

In [14]:
# Now multiplying the flux values by the relevant power for its GOES class
glux.loc[glux.goes_class == 'A', 'goes_flux'] *= 1e-8
glux.loc[glux.goes_class == 'B', 'goes_flux'] *= 1e-7
glux.loc[glux.goes_class == 'C', 'goes_flux'] *= 1e-6
glux.loc[glux.goes_class == 'M', 'goes_flux'] *= 1e-5
glux.loc[glux.goes_class == 'X', 'goes_flux'] *= 1e-4

In [15]:
# Now let's replace the NaNs 
glux.goes_flux = glux.goes_flux.replace({0:np.nan})
glux.goes_class = glux.goes_class.replace({0:np.nan})
glux.flare_type = glux.flare_type.replace({0:np.nan})

In [16]:
glux

goes_class  goes_flux flare_type
0            NaN        NaN        NaN
1            NaN        NaN        NaN
2            NaN        NaN        NaN
3            NaN        NaN        NaN
4            NaN        NaN        NaN
...          ...        ...        ...
30223        NaN        NaN        NaN
30224        NaN        NaN        NaN
30225        NaN        NaN        NaN
30226        NaN        NaN        NaN
30227        NaN        NaN        NaN

[30228 rows x 3 columns]

In [17]:
df

cme_time  cme_angle  cme_width  cme_speed  initial_2nd_order  \
0     1996-08-01 12:36:35      274.0       35.0      499.0              451.0   
1     1996-08-01 18:12:21      210.0       82.0      118.0               94.0   
2     1996-08-03 19:45:37      231.0       26.0       71.0               59.0   
3     1996-08-07 13:15:05      242.0       27.0        NaN                NaN   
4     1996-08-10 09:25:05       68.0       68.0      148.0              133.0   
...                   ...        ...        ...        ...                ...   
30223 2020-05-29 11:24:05       84.0       52.0      347.0              406.0   
30224 2020-05-29 15:12:05       83.0       46.0      204.0              215.0   
30225 2020-05-29 17:36:05       76.0       34.0      210.0              295.0   
30226 2020-05-30 01:25:43       80.0       39.0      269.0              203.0   
30227 2020-05-30 08:48:05      259.0       23.0      146.0              107.0   

       final_2nd_order  20r_2nd_order  cme_acceleration cme_mass  \
0                551.0          615.0               7.4  2.6e+14   
1                142.0          324.0               4.0  9.8e+13   
2                 84.0          128.0               0.5      NaN   
3                  NaN            NaN               NaN      NaN   
4                164.0          201.0               1.0  4.6e+14   
...                ...            ...               ...      ...   
30223            288.0            0.0             -21.9      NaN   
30224            193.0            0.0              -2.8      NaN   
30225            120.0            0.0             -19.3      NaN   
30226            339.0          331.0               2.8      NaN   
30227            180.0          399.0               6.5      NaN   

      cme_kinetic_energy  ...  smart_psl_length smart_r_value smart_b_max  \
0                3.2e+29  ...               NaN           NaN         NaN   
1                6.8e+27  ...               NaN           NaN         NaN   
2                    NaN  ...               NaN           NaN         NaN   
3                    NaN  ...               NaN           NaN         NaN   
4                5.0e+28  ...               NaN           NaN         NaN   
...                  ...  ...               ...           ...         ...   
30223                NaN  ...               NaN           NaN         NaN   
30224                NaN  ...               NaN           NaN         NaN   
30225                NaN  ...               NaN           NaN         NaN   
30226                NaN  ...               NaN           NaN         NaN   
30227                NaN  ...               NaN           NaN         NaN   

      smart_b_min smart_observation_time smart_string_latlon smart_total_area  \
0             NaN                    NaT                 NaN              NaN   
1             NaN                    NaT                 NaN              NaN   
2             NaN                    NaT                 NaN              NaN   
3             NaN                    NaT                 NaN              NaN   
4             NaN                    NaT                 NaN              NaN   
...           ...                    ...                 ...              ...   
30223         NaN                    NaT                 NaN              NaN   
30224         NaN                    NaT                 NaN              NaN   
30225         NaN                    NaT                 NaN              NaN   
30226         NaN                    NaT                 NaN              NaN   
30227         NaN                    NaT                 NaN              NaN   

      smart_negative_area smart_hg_longitude goes_class  
0                     NaN                NaN        NaN  
1                     NaN                NaN        NaN  
2                     NaN                NaN        NaN  
3                     NaN                NaN        NaN  
4                     NaN              

In [18]:
# Adding the new goes_flux column into the database
df['goes_flux'] = glux.goes_flux

In [19]:
df['goes_flux']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
30223   NaN
30224   NaN
30225   NaN
30226   NaN
30227   NaN
Name: goes_flux, Length: 30228, dtype: float64

### SRS Hale Classes

In [20]:
# Need to group Beta and BETA together, etc.
df['srs_hale'].replace({"ALPHA": 1, "Alpha": 1, 
                         "BETA": 2, "Beta": 2, 
                         "BETA-GAMMA": 3, "Beta-Gamma": 3, 
                         "BETA-DELTA": 4, "Beta-Delta": 4, 
                         "BETA-GAMMA-DELTA": 5, "Beta-Gamma-Delta": 5,
                         "GAMMA-DELTA": 6, "Gamma-Delta": 6,}, inplace=True)

In [21]:
df['srs_hale']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
30223   NaN
30224   NaN
30225   NaN
30226   NaN
30227   NaN
Name: srs_hale, Length: 30228, dtype: float64

### CME halo

In [22]:
# Replacing the non-numeric hale classes with numeric values
df['cme_halo'].replace({"I": "1", "II": "2", "III": "3", "IV": "4"}, inplace=True)
df['cme_halo'] = df['cme_halo'].astype(float)

In [23]:
df['cme_halo']

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
30223    1.0
30224    1.0
30225    1.0
30226    1.0
30227    1.0
Name: cme_halo, Length: 30228, dtype: float64

### Blank Entries

In [24]:
pd.set_option('display.max_rows', 10) #for all rows --> None
print((df == '').sum(axis=0))
sum((df == '').sum(axis=0))

cme_time               0
cme_angle              0
cme_width              0
cme_speed              0
initial_2nd_order      0
                      ..
smart_total_area       0
smart_negative_area    0
smart_hg_longitude     0
goes_class             0
goes_flux              0
Length: 61, dtype: int64


9195

#### Blanks:
- cme_mass: 1
- cme_kinetic_energy: 1
- cme_remarks: 9193               

In [25]:
# Replacing blanks with NaNs
df.cme_mass = df.cme_mass.replace({'':np.nan})
df.cme_remarks = df.cme_remarks.replace({'':np.nan})
df.cme_kinetic_energy = df.cme_kinetic_energy.replace({'':np.nan})

In [26]:
# This should be zero now
sum((df == '').sum(axis=0))

0

### Reformatting some dtypes for ease of use during the project

In [27]:
df['srs_area'] = df['srs_area'].astype(float)
df['cme_mass'] = df['cme_mass'].astype(float)
df['srs_no_spots'] = df['srs_no_spots'].astype(float)
df['smart_total_flux'] = df['smart_total_flux'].astype(float)
df['cme_kinetic_energy'] = df['cme_kinetic_energy'].astype(float)
df['smart_positive_flux'] = df['smart_positive_flux'].astype(float)
df['smart_negative_flux'] = df['smart_negative_flux'].astype(float)

### NOTE: there are many zero values

In [33]:
pd.set_option('display.max_rows', 10) #for all rows --> None
print(sum((df == 0).sum(axis=0)))
(df == 0).sum(axis=0)

#.value_counts().to_dict()

8262


cme_time                 0
cme_angle               48
cme_width                0
cme_speed                0
initial_2nd_order      846
                      ... 
smart_total_area         0
smart_negative_area      4
smart_hg_longitude       1
goes_class               0
goes_flux                0
Length: 61, dtype: int64

#### Zero's that don't really make sense:
- initial 2nd order: 846
- final 2nd order: 88
- SRS area: 40
- SMART flux frac: 172
- SMART WLSG: 91
- SMART PSL: 122

Total = 1359

Just going to remove those I am 100% sure about

In [33]:
# SRS area
(df.srs_area == 0).astype(float).sum(axis=0)

40.0

In [34]:
# Setting the zero values as NaNs 
df.srs_area = df.srs_area.replace({0:np.nan})

# Checking
(df.srs_area == 0).astype(float).sum(axis=0)

0.0

In [35]:
# WLSG and PIL --> decided to leave the 91 (WLSG) and 122 (PIL) zero values as there's issues with these properties

"""
# WLSG 
(df.smart_wlsg == 0).astype(float).sum(axis=0)
df.smart_wlsg = df.srs_area.replace({0:np.nan})

# PIL 
(df.smart_psl_length == 0).astype(float).sum(axis=0)
df.smart_psl_length = df.srs_area.replace({0:np.nan})
"""

91.0

## Fixing the issue with the LMSAL entries in the flare_end_time column

In [35]:
# Getting columns I want
df1 = df[['flare_start_time', 'flare_end_time']]
df1 = df1.dropna(how='any')
df1 = df1.reset_index()

In [36]:
def calculate_flare_duration(data_start, data_end):
    """Get flare duration in minutes"""
    data_out = data_end - data_start
    for i in range(len(data_out)):
        try:
            data_out[i] = (data_out[i]).total_seconds()/60.
        except AttributeError:
            continue
    return data_out

In [37]:
# Finding the duration of the flares
durations = calculate_flare_duration(df1['flare_start_time'], df1['flare_end_time'])

In [38]:
# Finding out how many are negative
print((durations < 0).astype(int).sum(axis=0))
print((durations > 0).astype(int).sum(axis=0))

784
15768


In [39]:
# Making a list of the indicies for the incorrect entries 
wrong_ind = df1.index[durations < 0].tolist()

In [40]:
# Checking that the length matches up
len(wrong_ind)

784

In [41]:
# Checking which database the wrong entries come from
nd = df[['flare_start_time', 'flare_end_time', 'flare_type']]
nd = nd.dropna(how='any')
nd = nd.reset_index(drop=True)

In [42]:
# Making a df containing the indicies of the incorrect entries
wrong_db = nd.iloc[wrong_ind, :]
wrong_db

flare_start_time      flare_end_time flare_type
2860  2002-10-13 23:42:00 2002-10-13 01:08:00      lmsal
2861  2002-10-13 23:42:00 2002-10-13 01:08:00      lmsal
2862  2002-10-13 23:42:00 2002-10-13 01:08:00      lmsal
2863  2002-10-14 14:45:00 2002-10-14 04:55:00      lmsal
2871  2002-10-16 22:20:00 2002-10-16 01:00:00      lmsal
...                   ...                 ...        ...
16476 2018-05-03 23:59:00 2018-05-03 00:16:00      lmsal
16494 2019-01-03 20:49:00 2019-01-03 04:16:00      lmsal
16516 2019-04-12 22:36:00 2019-04-12 00:30:00      lmsal
16544 2019-05-08 23:45:00 2019-05-08 00:08:00      lmsal
16545 2019-05-08 23:45:00 2019-05-08 00:08:00      lmsal

[784 rows x 3 columns]

In [43]:
# Checking what databases are giving the wrong entries
ct = wrong_db['flare_type'].value_counts().to_dict()
ct

{'lmsal': 784}

In [44]:
# Okay, now let's ttry to replace the incorrect values
df1['index']

0           70
1           71
2           72
3           79
4           80
         ...  
16547    29698
16548    29713
16549    29773
16550    29785
16551    29951
Name: index, Length: 16552, dtype: int64

In [45]:
# Putting the ORIGINAL indices of the wrong entries into a list
correct_ind = df1['index'].iloc[wrong_ind].tolist()
correct_ind

[6759,
 6760,
 6761,
 6762,
 6773,
 6792,
 6914,
 6917,
 6932,
 6933,
 6934,
 6936,
 6937,
 7023,
 7024,
 7027,
 7052,
 7053,
 7059,
 7166,
 7342,
 7344,
 7350,
 7362,
 7406,
 7407,
 7409,
 7415,
 7416,
 7417,
 7418,
 7504,
 7505,
 7508,
 7509,
 7531,
 7728,
 7736,
 7737,
 7782,
 7795,
 7819,
 7827,
 7845,
 7865,
 7866,
 7868,
 7921,
 8002,
 8003,
 8004,
 8005,
 8044,
 8045,
 8153,
 8154,
 8676,
 8743,
 8744,
 8765,
 8769,
 8770,
 8779,
 8825,
 8900,
 8902,
 8911,
 8915,
 8917,
 8961,
 8962,
 8963,
 8998,
 9000,
 9001,
 9002,
 9015,
 9065,
 9066,
 9067,
 9068,
 9071,
 9131,
 9245,
 9246,
 9247,
 9252,
 9253,
 9262,
 10610,
 10613,
 10614,
 10615,
 10830,
 10831,
 10872,
 10873,
 10951,
 10952,
 10953,
 11140,
 11141,
 11142,
 11143,
 11144,
 11145,
 11208,
 11209,
 11210,
 11273,
 11274,
 11275,
 11277,
 11278,
 11279,
 11296,
 11324,
 11325,
 11328,
 11339,
 11340,
 11342,
 11355,
 11356,
 11357,
 11358,
 11359,
 11360,
 11367,
 11368,
 11369,
 11590,
 11669,
 11670,
 11671,
 11672,
 

In [46]:
new_df = df.copy()

In [47]:
# Replacing the wrong entries
new_df.flare_end_time.iloc[correct_ind] = (df.flare_end_time + dt.timedelta(days=1))

C:\Users\Peter\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [48]:
# Testing
print(df['flare_end_time'].iloc[6759])
print(new_df['flare_end_time'].iloc[6759])

2002-10-13 01:08:00
2002-10-14 01:08:00


In [49]:
# Adding in a new column for the flare durations
new_df['flare_duration'] = calculate_flare_duration(new_df['flare_start_time'], new_df['flare_end_time'])

In [50]:
jt = new_df['flare_duration']
jt = jt.dropna(how='any')
jt

70       33
71        7
72        6
79       58
80        9
         ..
29698    16
29713     5
29773    49
29785     7
29951    50
Name: flare_duration, Length: 16552, dtype: object

### Saving the new df

In [51]:
new_df.to_pickle('custom_df2.p')

In [52]:
# Testing the save
df2 = 'C:/Users/Peter/py_projects/solar_stats/custom_df2.p'
test = pd.read_pickle(df2)

In [53]:
test

cme_time  cme_angle  cme_width  cme_speed  initial_2nd_order  \
0     1996-08-01 12:36:35      274.0       35.0      499.0              451.0   
1     1996-08-01 18:12:21      210.0       82.0      118.0               94.0   
2     1996-08-03 19:45:37      231.0       26.0       71.0               59.0   
3     1996-08-07 13:15:05      242.0       27.0        NaN                NaN   
4     1996-08-10 09:25:05       68.0       68.0      148.0              133.0   
...                   ...        ...        ...        ...                ...   
30223 2020-05-29 11:24:05       84.0       52.0      347.0              406.0   
30224 2020-05-29 15:12:05       83.0       46.0      204.0              215.0   
30225 2020-05-29 17:36:05       76.0       34.0      210.0              295.0   
30226 2020-05-30 01:25:43       80.0       39.0      269.0              203.0   
30227 2020-05-30 08:48:05      259.0       23.0      146.0              107.0   

       final_2nd_order  20r_2nd_order  cme_acceleration cme_mass  \
0                551.0          615.0               7.4  2.6e+14   
1                142.0          324.0               4.0  9.8e+13   
2                 84.0          128.0               0.5      NaN   
3                  NaN            NaN               NaN      NaN   
4                164.0          201.0               1.0  4.6e+14   
...                ...            ...               ...      ...   
30223            288.0            0.0             -21.9      NaN   
30224            193.0            0.0              -2.8      NaN   
30225            120.0            0.0             -19.3      NaN   
30226            339.0          331.0               2.8      NaN   
30227            180.0          399.0               6.5      NaN   

       cme_kinetic_energy  ...  smart_b_max smart_b_min  \
0            3.200000e+29  ...          NaN         NaN   
1            6.800000e+27  ...          NaN         NaN   
2                     NaN  ...          NaN         NaN   
3                     NaN  ...          NaN         NaN   
4            5.000000e+28  ...          NaN         NaN   
...                   ...  ...          ...         ...   
30223                 NaN  ...          NaN         NaN   
30224                 NaN  ...          NaN         NaN   
30225                 NaN  ...          NaN         NaN   
30226                 NaN  ...          NaN         NaN   
30227                 NaN  ...          NaN         NaN   

      smart_observation_time  smart_string_latlon smart_total_area  \
0                        NaT                  NaN              NaN   
1                        NaT                  NaN              NaN   
2                        NaT                  NaN              NaN   
3                        NaT                  NaN              NaN   
4                        NaT                  NaN              NaN   
...                      ...                  ...              ...   
30223                    NaT                  NaN              NaN   
30224                    NaT                  NaN              NaN   
30225                    NaT                  NaN              NaN   
30226                    NaT                  NaN              NaN   
30227                    NaT                  NaN              NaN   

      smart_negative_area smart_hg_longitude goes_class goes_flux  \
0                     NaN                NaN        NaN       NaN   
1                     NaN                NaN        NaN       NaN   
2                     NaN                NaN        NaN       NaN   
3                     NaN                NaN        NaN       NaN   
4                     NaN                NaN        NaN       NaN   
...                   ...                ...        ...       ...   
30223                 NaN                NaN        NaN       NaN   
30224                 NaN                NaN        NaN       NaN   
30225                 NaN                NaN        NaN 